In [1]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog as tkFileDialog
from tkinter.filedialog import askdirectory
from PIL import Image,ImageTk, ImageFilter
import numpy as np
from skimage.util import random_noise
import cv2
from scipy.ndimage import convolve as conv
from scipy.ndimage import correlate as corr
import pydicom
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib, sys
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class MyWindow(tk.Frame):
    def __init__(self, *args, **kwargs):
        tk.Frame.__init__(self, *args, **kwargs)
        self.btn2=Button(self, text='noise', command=self.noise)
        self.b1=Button(self, text='rotate', command=self.rotate)
        self.b2=Button(self, text='filter', command=self.filtre)
        self.b3=Button(self, text='browse', command=self.enter)
        self.b4=Button(self, text='Save', command=self.save)
        self.b5=Button(self, text='compare', command=self.compare)
        self.b6=Button(self, text='save_as_dcm', command=self.save_dicom)
        self.b6.place(x=10,y=525)
        self.b5.place(x=300, y=560)
        self.b3.place(x=100, y=560)
        self.b1.place(x=250, y=525)
        self.b4.place(x=200, y=560)
        self.b2.place(x=350, y=525)
        self.btn2.place(x=150, y=525)
        
    def dcompare(self):
        def comp():
            try:
                if self.count==1:
                    True
            except:
                self.count=0
            self.n=int(sp.get())
            if self.count==1:
                self.comp.destroy()
            else:
                self.count=1
                for _ in range(self.n):
                    ifile = tkFileDialog.askopenfile(parent=self.comp,mode='rb',title='Choose a file')
                    path = ifile.name
                    image=Image.open(path)
                    w,h=image.size
                    image=image.resize((int(w/2),int(h/2)))
                    render=ImageTk.PhotoImage(image)
                    self.com_b.configure(text='exit')
                    self.label_com = Label(self.comp, image=render)
                    self.label_com.image=render
                    self.label_com.pack()
        try:
            self.text.pack_forget()
        except:
            True
        try:
            self.hello.destroy()
        except:
            True
        try:
            self.root.destroy()
        except:
            True
        try:
            self.nose.destroy()
        except:
            True
        try:
            self.fil.destroy()
        except:
            True
        self.comp=Toplevel(self)
        sp = Spinbox(self.comp, from_= 1, to = 4)
        sp.pack()
        v=Scrollbar(self.comp)
        v.pack(side=RIGHT, fill=Y)
        self.com_b=Button(self.comp, text='enter', command=comp)
        self.com_b.pack()
        
    def save_dicom(self):
        filename = tkFileDialog.asksaveasfile(mode='w', defaultextension=".dcm")
        if not filename:
             return
        
    def denter(self):
        ifile = tkFileDialog.askopenfile(parent=self,mode='rb',title='Choose a file')
        path = ifile.name
        im=pydicom.dcmread(path)
        photo=np.array(im.pixel_array)
        photo=np.uint8(cv2.normalize(photo, None, 0, 255, cv2.NORM_MINMAX))
        photo=cv2.equalizeHist(photo)
        self.image_new=Image.frombytes('L', (photo.shape[1],photo.shape[0]), photo.astype('b').tostring())
        self.render = ImageTk.PhotoImage(image=self.image_new) 
        l=Label(self,image=self.render)
        l.image=self.render
        l.pack()
        w,h=self.image_new.size
        self.image_new=self.image_new.resize((int(w/2),int(h/2)))
        
    def drotate(self):
        def rot():
            self.root.title("angle:"+str(sp.get()))
            imi=self.image_new.copy()
            imi = imi.rotate(int(sp.get()))
            self.rot_im=imi
            r0t = ImageTk.PhotoImage(imi)
            label_rot.configure(image=r0t)
            label_rot.image = r0t
            self.to_save=imi
        try:
            self.text.pack_forget()
        except:
            True
        try:
            self.hello.destroy()
        except:
            True
        try:
            self.comp.destroy()
        except:
            True
        try:
            self.nose.destroy()
        except:
            True
        try:
            self.fil.destroy()
        except:
            True
        self.root=Toplevel(self)
        sp = Spinbox(self.root, from_= 0, to = 360)
        sp.pack()
        render=ImageTk.PhotoImage(self.image_new)
        label_rot = Label(self.root,image=render)
        label_rot.pack()
        b3=Button(self.root, text='enter', command=rot).pack()
    
    def dnoise(self):
        def rota():
            self.joke=self.rot_im.copy()
            render=ImageTk.PhotoImage(self.joke)
            label_no.configure(image=render)
            label_no.image=render
        def normal():
            self.joke=self.image_new.copy()
            render=ImageTk.PhotoImage(self.joke)
            label_no.configure(image=render)
            label_no.image=render
        def no_amount():
            global amount
            try:
                tex_l.pack()
                key.pack()
            except:
                True
            tex_l.configure(text='choose proportion to be replaced')
            amount=Spinbox(self.nose,text='proportion to be replaced',format="%.2f",increment=0.01, from_=0,to=1)
            amount.pack()
            key.configure(text='enter', command=nono)
        def no_MnV():
            global mean,var
            try:
                tex_l.pack()
                key.pack()
            except:
                True
            tex_l.configure(text='choose mean and variance respectively')
            mean=Spinbox(self.nose,from_=0,to=10)
            mean.pack()
            var=Spinbox(self.nose,format="%.2f",increment=0.01, from_=0,to=1)
            var.pack()
            key.configure(text='enter', command=nono)
        def nono():
            tex_l.pack_forget()
            key.pack_forget()
            imi=self.joke
            imag=np.asarray(imi)
            photo=imag.copy()
            mode=v1.get()
            if ((mode=='s&p') or (mode=='salt') or (mode=='pepper')):
                a=amount.get()
                sp=random_noise(photo,mode=mode, amount=np.float(a), clip=True)
            
            elif ((mode=='gaussian')or(mode=='speckle')):
                m=np.int(mean.get())
                v=np.float(var.get())
                sp=random_noise(photo,mode=mode, mean=m,var=v, clip=True)
            else:
                sp=random_noise(photo,mode=mode, clip=True)
            data=sp
            data = data / data.max() #normalizes data in range 0 - 255
            data = 255 * data
            sp = data.astype(np.uint8)
            imi=Image.fromarray(sp)
            self.to_save=imi
            render=ImageTk.PhotoImage(imi)
            label_no.configure(image=render)
            label_no.image = render # keep a reference!
            try:
                amount.destroy()
            except:
                True
            try:
                mean.destroy()
                var.destroy()
            except:
                True
        self.nose=Toplevel(self)
        try:
            self.text.pack_forget()
        except:
            True
        try:
            self.comp.destroy()
        except:
            True
        try:
            self.root.destroy()
        except:
            True
        try:
            self.hello.destroy()
        except:
            True
        try:
            self.fil.destroy()
        except:
            True
        self.nose.title("noisy image")
        v1=StringVar()
        b1=Button(self.nose, text='apply noise on rotated image', command=rota).pack()
        b2=Button(self.nose, text='apply noise on original image', command=normal).pack()
        n1=Checkbutton(self.nose, text='s&p',command=no_amount, variable=v1,onvalue = 's&p', offvalue = 0).pack()
        n2=Checkbutton(self.nose,text='salt',command=no_amount, variable=v1,onvalue = 'salt', offvalue = 0).pack()
        n3=Checkbutton(self.nose,text='pepper',command=no_amount, variable=v1,onvalue = 'pepper', offvalue = 0).pack()
        n4=Checkbutton(self.nose, text='localvar',command=nono, variable=v1,onvalue = 'localvar', offvalue = 0).pack()
        n5=Checkbutton(self.nose,text='gaussian',command=no_MnV, variable=v1,onvalue = 'gaussian', offvalue = 0).pack()
        n6=Checkbutton(self.nose,text='poisson',command=nono, variable=v1,onvalue = 'poisson', offvalue = 0).pack()
        n7=Checkbutton(self.nose,text='speckle',command=no_MnV, variable=v1,onvalue = 'speckle', offvalue = 0).pack()
        b=Button(self.nose, text='filter this image', command=self.filtre).pack()
        bfi=Button(self.nose, text='Save_jpg', command=self.save).pack()
        label_no=Label(self.nose,image=None)
        label_no.pack()
        tex_l=Label(self.nose,text='hello')
        tex_l.pack()
        key=Button(self.nose, text="I'll work later")
        key.pack()
    
    def dfiltre(self):
        def rota():
            self.x=self.rot_im.copy()
            render=ImageTk.PhotoImage(self.x)
            label_fil.configure(image=render)
            label_fil.image=render
            self.x=np.asarray(self.x)
        def normal():
            self.x=self.image_new.copy()
            render=ImageTk.PhotoImage(self.x)
            label_fil.configure(image=render)
            label_fil.image=render
            self.x=np.asarray(self.x)
        def noisy_im():
            self.x=self.to_save.copy()
            render=ImageTk.PhotoImage(self.x)
            label_fil.configure(image=render)
            label_fil.image=render
            self.x=np.asarray(self.x)
        def filtering():
                filt=str(v1.get())
                if filt=='gaus':
                    try:
                        tex_l.pack()
                        key.pack()
                    except:
                        True
                    tex_l.configure(text='choose: size of kernel and sigma')
                    self.sp=Spinbox(self.fil,from_=0,to=10)
                    self.sp.pack()
                    self.sigma=Spinbox(self.fil,format="%.1f",increment=0.1,from_=0,to=10)
                    self.sigma.pack()
                    self.r1=Radiobutton(self.fil, text='convule', variable=self.c, value='conv')
                    self.r2=Radiobutton(self.fil, text='correlate', variable=self.c, value='corr')
                    self.r1.pack()
                    self.r2.pack()
                    key.configure(text='filtre',command=gaussian)
                elif filt=='avg':
                    try:
                        tex_l.pack()
                        key.pack()
                    except:
                        True
                    tex_l.configure(text='kernel size:')
                    self.sp=Spinbox(self.fil,from_=0,to=100)
                    self.sp.pack()
                    self.r1=Radiobutton(self.fil, text='convule', variable=self.c, value='conv')
                    self.r2=Radiobutton(self.fil, text='correlate', variable=self.c, value='corr')
                    self.r1.pack()
                    self.r2.pack()
                    key.configure(text='filtre',command=average)
                elif filt=='med':
                    try:
                        tex_l.pack()
                        key.pack()
                    except:
                        True
                    tex_l.configure(text='')
                    key.configure(text='filtre',command=median)
                elif filt=='bil':
                    try:
                        tex_l.pack()
                        key.pack()
                    except:
                        True
                    tex_l.configure(text='choose: diameter,sigmaColor and sigmaSpace')
                    self.sp=Spinbox(self.fil,from_=0,to=10)
                    self.sp.pack()
                    self.sigma=Spinbox(self.fil,from_=10,to=150)
                    self.sigma.pack()
                    self.sigma1=Spinbox(self.fil,from_=10,to=150)
                    self.sigma1.pack()
                    key.configure(text='filtre',command=bilateral)
                elif filt=='lap':
                    try:
                        tex_l.pack()
                        key.pack()
                    except:
                        True
                    tex_l.configure(text='')
                    key.configure(text='filtre',command=laplacian)
                elif filt=='sob':
                    try:
                        tex_l.pack()
                        key.pack()
                    except:
                        True
                    tex_l.configure(text='choose axis:')
                    self.r1=Radiobutton(self.fil, text='x-axis', variable=self.c, value='x')
                    self.r2=Radiobutton(self.fil, text='y-axis', variable=self.c, value='y')
                    self.r1.pack()
                    self.r2.pack()
                    key.configure(text='filtre',command=sobel)
                else:
                    False
        
        def  gaussian():
            tex_l.pack_forget()
            key.pack_forget()
            size=int(self.sp.get())
            sigma=np.float(self.sigma.get())
            self.sp.destroy()
            self.sigma.destroy()
            typ=self.c.get()
            self.r1.pack_forget()
            self.r2.pack_forget()
            size=(size,size)
            m,n = [(ss-1.)/2. for ss in size]
            y,x = np.ogrid[-m:m+1,-n:n+1]
            h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
            h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
            sumh = h.sum()
            if sumh != 0:
                h /= sumh
            a=cv2.split(self.x)
            if typ=='conv':
                a[0]=conv(a[0], h, mode='nearest')
            else:
                a[0]=corr(a[0], h, mode='nearest')
            self.x=a[0]
            imi=Image.fromarray(self.x)
            self.to_save_f=imi
            render=ImageTk.PhotoImage(imi)
            label_new.configure(image=render)
            label_new.image=render
        def average():
            tex_l.pack_forget()
            key.pack_forget()
            size=int(self.sp.get())
            self.sp.destroy()
            h=np.ones((size,size))
            h[:,:]=1/(size*size)
            a=cv2.split(self.x)
            typ=self.c.get()
            self.r1.pack_forget()
            self.r2.pack_forget()
            if typ=='conv':
                a[0]=conv(a[0], h, mode='nearest')
            else:
                a[0]=corr(a[0], h, mode='nearest')
            self.x=a[0]
            imi=Image.fromarray(self.x)
            self.to_save_f=imi
            render=ImageTk.PhotoImage(imi)
            label_new.configure(image=render)
            label_new.image=render
        def median():
            tex_l.pack_forget()
            key.pack_forget()
            median_im=Image.fromarray((self.x*255).astype(np.uint8)).filter(ImageFilter.MedianFilter(size = 3))
            self.to_save_f=median_im
            render=ImageTk.PhotoImage(median_im)
            label_new.configure(image=render)
            label_new.image=render
        def bilateral():
            tex_l.pack_forget()
            key.pack_forget()
            size=int(self.sp.get())
            self.sp.destroy()
            sigma=int(self.sigma.get())
            sigma2=int(self.sigma1.get())
            self.sigma.destroy()
            self.sigma1.destroy()
            a=cv2.split(self.x)
            im_bil1=cv2.bilateralFilter(self.x,size,sigma,sigma2)
            imi=Image.fromarray(im_bil1)
            self.to_save_f=imi
            render=ImageTk.PhotoImage(imi)
            label_new.configure(image=render)
            label_new.image=render
        def laplacian():
            tex_l.pack_forget()
            key.pack_forget()
            a=cv2.split(self.x) 
            a[0]=cv2.Laplacian(a[0],cv2.CV_16S,3)
            a[0] =cv2.convertScaleAbs(a[0])
            self.x=a[0]
            imi=Image.fromarray(self.x)
            self.to_save_f=imi
            render=ImageTk.PhotoImage(imi)
            label_new.configure(image=render)
            label_new.image=render
        def sobel():
            tex_l.pack_forget()
            key.pack_forget()
            a=cv2.split(self.x)
            axis=self.c.get()
            self.r1.pack_forget()
            self.r2.pack_forget()
            if axis=='x':
                x=1
                y=0
            else:
                x=0
                y=1
            a[0] = cv2.Sobel(a[0],cv2.CV_16S,x,y,ksize=-1)
            a[0] =cv2.convertScaleAbs(a[0])
            self.x=a[0]
            imi=Image.fromarray(self.x)
            self.to_save_f=imi
            render=ImageTk.PhotoImage(imi)
            label_new.configure(image=render)
            label_new.image=render
            
        self.fil=Toplevel(self)
        try:
            self.text.pack_forget()
        except:
            True
        try:
            self.root.destroy()
        except:
            True
        try:
            self.comp.destroy()
        except:
            True
        try:
            self.hello.destroy()
        except:
            True
        try:
            self.nose.destroy()
        except:
            True
        self.fil.title("filter")
        v1=StringVar()
        noise=True
        self.c=StringVar()
        n1=Checkbutton(self.fil, text='choose:noisy image',command=self.noise, variable=v1,onvalue = 'yes', offvalue = 0).pack()
        b1=Button(self.fil, text='filter rotated image', command=rota).pack()
        b2=Button(self.fil, text='filter original image', command=normal).pack()
        b3=Button(self.fil, text='filter noisy image', command=noisy_im).pack()
        label_fil=Label(self.fil,image=None)
        label_fil.pack()
        f1=Checkbutton(self.fil, text='gaussian kernel',command=filtering, variable=v1, onvalue='gaus', offvalue=0).pack()
        f2=Checkbutton(self.fil, text='average kernel',command=filtering, variable=v1, onvalue='avg', offvalue=0).pack()
        f3=Checkbutton(self.fil, text='median kernel',command=filtering, variable=v1, onvalue='med', offvalue=0).pack()
        f4=Checkbutton(self.fil, text='bilateral kernel',command=filtering, variable=v1, onvalue='bil', offvalue=0).pack()
        f5=Checkbutton(self.fil, text='laplacian kernel',command=filtering, variable=v1, onvalue='lap', offvalue=0).pack()
        f6=Checkbutton(self.fil, text='sobel kernel',command=filtering, variable=v1, onvalue='sob', offvalue=0).pack()
        tex_l=Label(self.fil,text='hello')
        tex_l.pack()
        key=Button(self.fil, text="I'll work later")
        key.pack()
        label_new=Label(self.fil,image=None)
        label_new.pack()
        bfi=Button(self.fil, text='Save_jpg', command=self.save_f)
        bfi.pack()

window=tk.Tk()
mywin=MyWindow(window)
mywin.pack(side="top", fill="both", expand=True)
window.title('Image Processing toolkit')
window.geometry("450x600+10+10")
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/saumyakansal/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-12720f26ccd8>", line 166, in rota
    label_np.image=render
NameError: name 'label_np' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/saumyakansal/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-12720f26ccd8>", line 171, in normal
    label_np.image=render
NameError: name 'label_np' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/saumyakansal/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-12720f26ccd8>", line 166, in rota
    label_np.image=render
NameError: name 'label_np' is not defined
Exception in Tkinter callback
Trace